## Converting the zju datasets into sfm for gaussian splatting

In [1]:
# All imports here
import os
from parameter_reader import CameraParametersExtractor
from database import COLMAPDatabase, blob_to_array

Given: 
- intri.yml
- extri.yml 

In [2]:
# The File Struture Should Be 
'''
/path/to/your/
│
├── intri.yml
├── entri.yml
│
├── inputs/      # Base directory for all image groups
│   ├── camera_grp1/  # Images from camera group 1
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   │
│   ├── camera_grp2/  # Images from camera group 2
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   │
│   └── ...      # More groups as needed
|
├── images/      # Will be created by this code
│   ├── camera_grp1/  # Images from camera group 1
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   │
│   ├── camera_grp2/  # Images from camera group 2
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   │
│   └── ...      # More groups as needed
│
└── sparse/      # Will be created by this code 
    ├── cameras.txt 
    ├── images.txt
    ├── points3D.txt
    └── 0/
        ├── cameras.bin
        ├── images.bin
        └── points3D.bin

'''


'\n/path/to/your/\n│\n├── intri.yml\n├── entri.yml\n│\n├── inputs/      # Base directory for all image groups\n│   ├── camera_grp1/  # Images from camera group 1\n│   │   ├── image1.jpg\n│   │   ├── image2.jpg\n│   │   └── ...\n│   │\n│   ├── camera_grp2/  # Images from camera group 2\n│   │   ├── image1.jpg\n│   │   ├── image2.jpg\n│   │   └── ...\n│   │\n│   └── ...      # More groups as needed\n|\n├── images/      # Will be created by this code\n│   ├── camera_grp1/  # Images from camera group 1\n│   │   ├── image1.jpg\n│   │   ├── image2.jpg\n│   │   └── ...\n│   │\n│   ├── camera_grp2/  # Images from camera group 2\n│   │   ├── image1.jpg\n│   │   ├── image2.jpg\n│   │   └── ...\n│   │\n│   └── ...      # More groups as needed\n│\n└── sparse/      # Will be created by this code \n    ├── cameras.txt \n    ├── images.txt\n    ├── points3D.txt\n    └── 0/\n        ├── cameras.bin\n        ├── images.bin\n        └── points3D.bin\n\n'

#### Set Paths and create Directories

In [3]:
# Set the paths here
rootdir = "data/mocap_small"
sparsedir = f"{rootdir}/sparse"
imagedir = f"{rootdir}/inputs"
database_path = f"{rootdir}/database.db"
outputdir = f"{sparsedir}/0"
# Create folders and files here
# These are been created according to https://colmap.github.io/faq.html#reconstruct-sparse-dense-model-from-known-camera-poses
# Templates for all files are explained at https://colmap.github.io/format.html#output-format

def create_empty_file(filepath):
    with open(filepath, 'w') as file:
        pass  # No need to write anything, just creating the file

os.makedirs(outputdir, exist_ok=True)
create_empty_file(f"{sparsedir}/points3D.txt")


#### Run the feature Extractor

In [4]:
# Now we need need to run colmap feature extractor
command = f"\
    colmap feature_extractor \
    --database_path {rootdir}/database.db \
    --image_path {imagedir} \
    --SiftExtraction.use_gpu 0"
os.system(command)



Feature extraction

Processed file [1/552]
  Name:            00/000001.jpg
  Dimensions:      1024 x 1024
  Camera:          #2 - SIMPLE_RADIAL
  Focal Length:    1228.80px
  Features:        1463
Processed file [2/552]
  Name:            00/000000.jpg
  Dimensions:      1024 x 1024
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    1228.80px
  Features:        1426
Processed file [3/552]
  Name:            00/000005.jpg
  Dimensions:      1024 x 1024
  Camera:          #6 - SIMPLE_RADIAL
  Focal Length:    1228.80px
  Features:        1421
Processed file [4/552]
  Name:            00/000002.jpg
  Dimensions:      1024 x 1024
  Camera:          #3 - SIMPLE_RADIAL
  Focal Length:    1228.80px
  Features:        1428
Processed file [5/552]
  Name:            00/000004.jpg
  Dimensions:      1024 x 1024
  Camera:          #5 - SIMPLE_RADIAL
  Focal Length:    1228.80px
  Features:        1429
Processed file [6/552]
  Name:            00/000003.jpg
  Dimensions:      1024 x 1024
  

0

#### Use the database as a reference to create images.txt

In [16]:
# This parts creates the images.txt using the ids from database

obj = CameraParametersExtractor(f"{rootdir}/extri.yml")
db = COLMAPDatabase.connect(database_path)
images_list = db.execute("SELECT image_id, name FROM images").fetchall()


names = obj.get_names()
camera_params = {}
image_params = {}
imageslst = []
for name in names:
    param = obj.get_extrinsic_params(name)
    camera_params[name] = list(map(str,[param["QW"], param["QX"], param["QY"], param["QZ"], param["TX"], param["TY"], param["TZ"]]))

for image in images_list:
    image_id, img_path = image
    cam_id = img_path.split("/")[0]
    param = [cam_id]
    param.extend(camera_params[cam_id])
    image_params[image_id] =param
    imageslst.append(
        f"{image_id} {' '.join(camera_params[cam_id])} {name} {img_path}"
    )
    imageslst.append("")

with open(f"{sparsedir}/images.txt", 'w') as file:
    for item in imageslst:
        file.write(f"{item}\n")

# Column names: ['image_id', 'name', 'camera_id', 'prior_qw', 'prior_qx', 'prior_qy', 'prior_qz', 'prior_tx', 'prior_ty', 'prior_tz']
# Connect to the database
# db = COLMAPDatabase.connect(database_path)

# for img_path, (img_id, cam_id, qw, qx, qy, qz, tx, ty, tz) in image_params.items():
#     # Update the camera parameters
#     db.execute("UPDATE images SET image_id=?, camera_id=?, prior_qw=?, prior_qx=?, prior_qy=?, prior_qz=?, prior_tx=?, prior_ty=?, prior_tz=? WHERE name=?", 
#                 (img_id, cam_id, qw, qx, qy, qz, tx, ty, tz, img_path))

db.commit()
db.close()



#### Create Camera.txt 

In [5]:
# This parts creates the cameras.txt
# TODO - Read the intri.yml - Done
# TODO - Create a list with each line as CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[] - Done
# TODO - Save the text image - Done

int_obj = CameraParametersExtractor(f"{rootdir}/intri.yml")
names = int_obj.get_names()
camera_lst = []
for name in names:
    param = int_obj.get_intrinsic_params(name)
    camera_int = list(map(str,[name ,"SIMPLE_PINHOLE", 1024, 1024, param['focal_length'], param['px'], param['py']]))
    camera_lst.append(" ".join(camera_int))

with open(f"{sparsedir}/cameras.txt", 'w') as file:
    for item in camera_lst:
        file.write(f"{item}\n")

#### Copy the Contents of camera.txt to the database

In [6]:
# Using database.py to copy the camera's intrisic parameters
# The source of code - https://github.com/colmap/colmap/blob/main/scripts/python/database.py
from database import COLMAPDatabase, blob_to_array

# Adjust the paths according to your setup
db_path = f'{rootdir}/database.db'  # Path to your COLMAP database
cameras_txt_path = f'{sparsedir}/cameras.txt'  # Path to cameras.txt

# Function to read cameras.txt and return camera parameters
def read_cameras_txt(cameras_txt_path):
    camera_params = {}
    with open(cameras_txt_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) > 3:  # Simple check to skip incorrect lines
                camera_id = int(parts[0])
                model = parts[1]
                width = int(parts[2])
                height = int(parts[3])
                params = list(map(float, parts[4:]))
                camera_params[camera_id] = (model, width, height, params)
    return camera_params

camera_params = read_cameras_txt(cameras_txt_path)
    
def update_camera_parameters(db_path, camera_params):
    # Connect to the database
    db = COLMAPDatabase.connect(db_path)
    
    for camera_id, (model, width, height, params) in camera_params.items():
        # Convert parameters list to bytes
        params_bytes = COLMAPDatabase.add_camera(
            model=model,
        width=width,
        height=height,
        params=params,
        camera_id=camera_id
        )
        # Update the camera parameters
        db.execute("UPDATE cameras SET model=?, width=?, height=?, params=? WHERE camera_id=?", 
                   (model, width, height, params_bytes, camera_id))
    
    db.commit()
    db.close()

#### Run the feature Extractor

In [8]:
# Running the feaure mapper
command = f"colmap exhaustive_matcher \
            --database_path {rootdir}/database.db \
            --SiftMatching.use_gpu 0"

os.system(command)


Exhaustive feature matching

Matching block [1/12, 1/12] in 7.445s
Matching block [1/12, 2/12] in 13.212s
Matching block [1/12, 3/12] in 7.801s
Matching block [1/12, 4/12] in 12.691s
Matching block [1/12, 5/12] in 14.141s
Matching block [1/12, 6/12] in 12.870s
Matching block [1/12, 7/12] in 9.865s
Matching block [1/12, 8/12] in 14.880s
Matching block [1/12, 9/12] in 16.168s
Matching block [1/12, 10/12] in 9.192s
Matching block [1/12, 11/12] in 13.332s
Matching block [1/12, 12/12] in 0.364s
Matching block [2/12, 1/12] in 10.502s
Matching block [2/12, 2/12] in 7.707s
Matching block [2/12, 3/12] in 7.649s
Matching block [2/12, 4/12] in 13.093s
Matching block [2/12, 5/12] in 14.069s
Matching block [2/12, 6/12] in 13.286s
Matching block [2/12, 7/12] in 12.909s
Matching block [2/12, 8/12] in 15.435s
Matching block [2/12, 9/12] in 16.033s
Matching block [2/12, 10/12] in 8.637s
Matching block [2/12, 11/12] in 14.230s
Matching block [2/12, 12/12] in 0.350s
Matching block [3/12, 1/12] in 10.183

0

#### Run the Image Undistorter 

In [37]:
commd = f"colmap image_undistorter \
    --image_path {imagedir}\
    --input_path {sparsedir} \
    --output_path {outputdir}"

os.system(commd)



Reading reconstruction

 => Reconstruction with 552 images and 0 points

Image undistortion

Undistorted image found; copying to location: data/mocap_small/images/00/000000.jpg
Undistorted image found; copying to location: Undistorted image found; copying to location: data/mocap_small/images/00/000005.jpgdata/mocap_small/images/00/000006.jpg

Undistorted image found; copying to location: data/mocap_small/images/00/000003.jpg
Undistorted image found; copying to location: Undistorted image found; copying to location: data/mocap_small/images/00/000012.jpg
Undistorted image found; copying to location: Undistorted image found; copying to location: data/mocap_small/images/00/000015.jpg
data/mocap_small/images/00/000008.jpgUndistorted image found; copying to location: Undistorted image found; copying to location: Undistorted image found; copying to location: data/mocap_small/images/00/000007.jpgUndistorted image found; copying to location: Undistorted image found; copying to location: 
Undis

0

#### Extra Code 

In [38]:
command = f"colmap point_triangulator \
    --database_path {rootdir}/database.db \
    --image_path {imagedir}\
    --input_path  {sparsedir}\
    --output_path {sparsedir}"
os.system(command)


Loading model


Loading database

Loading cameras... 552 in 0.000s
Loading matches... 138824 in 0.235s
Loading images... 552 in 0.023s (connected 552)
Building correspondence graph... in 1.375s (ignored 0)

Elapsed time: 0.027 [minutes]


Triangulating image #549 (0)

  => Image sees 0 / 952 points
  => Triangulated 658 points

Triangulating image #548 (1)

  => Image sees 546 / 954 points
  => Triangulated 152 points

Triangulating image #540 (2)

  => Image sees 642 / 959 points
  => Triangulated 106 points

Triangulating image #550 (3)

  => Image sees 685 / 941 points
  => Triangulated 81 points

Triangulating image #552 (4)

  => Image sees 726 / 970 points
  => Triangulated 70 points

Triangulating image #551 (5)

  => Image sees 731 / 939 points
  => Triangulated 55 points

Triangulating image #545 (6)

  => Image sees 745 / 924 points
  => Triangulated 36 points

Triangulating image #538 (7)

  => Image sees 761 / 937 points
  => Triangulated 38 points

Triangulating image #53

0

In [ ]:
mapper_cmd = ("colmap" + " mapper \
        --database_path " + rootdir + "/database.db \
        --image_path "  +  imagedir + \
        " --output_path "  + rootdir + "/sparse \
        --Mapper.ba_global_function_tolerance=0.000001")
os.system(mapper_cmd)

In [28]:
from database import COLMAPDatabase, blob_to_array
import numpy as np

# Specify the path to your COLMAP database
database_path = f'{rootdir}/database.db'

# Connect to the database
db = COLMAPDatabase.connect(database_path)

# Execute a query to fetch all cameras
images = db.execute("SELECT * FROM images").fetchall()

images = db.execute("SELECT image_id, name, camera_id, prior_qw, prior_qx FROM images").fetchall()

for image in images:
    image_id, name, camera_id, qw, qx = image
    # print(f"Image ID: {image_id}, Name: {name}, Camera ID: {camera_id}, qw:{qw}, qx:{qx}")


In [ ]:
cursor = db.execute('SELECT * FROM images LIMIT 0')
column_names = [desc[0] for desc in cursor.description]
print("Column names:", column_names)

Column names: ['image_id', 'name', 'camera_id', 'prior_qw', 'prior_qx', 'prior_qy', 'prior_qz', 'prior_tx', 'prior_ty', 'prior_tz']


In [31]:
db.commit()
db.close()



In [48]:
def read_extrinsics_text(path):
    """
    Taken from https://github.com/colmap/colmap/blob/dev/scripts/python/read_write_model.py
    """
    images = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                image_id = int(elems[0])
                qvec = np.array(tuple(map(float, elems[1:5])))
                tvec = np.array(tuple(map(float, elems[5:8])))
                camera_id = int(elems[8])
                image_name = elems[9]
                elems = fid.readline().split()
                xys = np.column_stack([tuple(map(float, elems[0::3])),
                                       tuple(map(float, elems[1::3]))])
                point3D_ids = np.array(tuple(map(int, elems[2::3])))
                images[image_id] = Image(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)
    return images


In [53]:
read_extrinsics_text(f"{rootdir}/output/images.bin")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xaf in position 12: invalid start byte